# 1) Importing Python Packages for GAN

In [2]:
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Reshape
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Dropout
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
!mkdir generated_images

# 2) Parameters for Neural Networks & Data

In [7]:
# cifar10'dan gelen imajlar 32x32 olduğu için 
img_width = 32
img_height = 32
# rgb olduğu için 3 kanal
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(learning_rate=0.0002)

# 3) Building Generator

In [17]:
def build_generator():
    model = Sequential()
    model.add(Dense(256 * 4 * 4, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((4,4,256)))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))

    model.summary()
    return model

generator = build_generator()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 4096)              413696    
                                                                 
 leaky_re_lu_23 (LeakyReLU)  (None, 4096)              0         
                                                                 
 reshape_7 (Reshape)         (None, 4, 4, 256)         0         
                                                                 
 conv2d_transpose_16 (Conv2D  (None, 8, 8, 128)        524416    
 Transpose)                                                      
                                                                 
 leaky_re_lu_24 (LeakyReLU)  (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose_17 (Conv2D  (None, 16, 16, 128)      262272    
 Transpose)                                           

# 4) Building Discriminator

In [28]:
def build_discriminator():
  # alexnet architecture
  model = Sequential()
  model.add(Conv2D(64, (3,3), padding='same', input_shape=img_shape))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(128, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(128, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(256, (3,3), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  # squishes to one dimension (binary output)
  model.add(Flatten())
  # drops %40 percent of neurons and increases stability, generalization and decreases overfitting
  model.add(Dropout(0.4))
  # output layer, output is zero or one
  model.add(Dense(1, activation='sigmoid'))

  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 leaky_re_lu_45 (LeakyReLU)  (None, 32, 32, 64)        0         
                                                                 
 conv2d_21 (Conv2D)          (None, 32, 32, 128)       73856     
                                                                 
 leaky_re_lu_46 (LeakyReLU)  (None, 32, 32, 128)       0         
                                                                 
 conv2d_22 (Conv2D)          (None, 32, 32, 128)       147584    
                                                                 
 leaky_re_lu_47 (LeakyReLU)  (None, 32, 32, 128)       0         
                                                                 
 conv2d_23 (Conv2D)          (None, 32, 32, 256)     

# 5) Connecting Neural Network to 

In [29]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer='adam')

# 6)

In [30]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
  r, c = 5, 5
  noise = np.random.normal(0, 1, (r * c, latent_dim))
  gen_imgs = generator.predict(noise)
  global save_name
  save_name += 0.00000001
  print("%.8f" % save_name)

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
    for j in range(c):
      axs[i,j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
      axs[i,j].axis('off')
      cnt += 1
  fig.savefig("generated_images/%.8f.png" % save_name)
  print('saved')
  plt.close()

# 7) Training GAN

In [ ]:
def train(epochs, batch_size=64, save_interval=200):
    (X_train, _), (_, _) = cifar10.load_data()

    # print(X_train.shape)
    # Rescale data between -1 and 1
    X_train = X_train / 127.5 -1.
    bat_per_epo = int(X_train.shape[0] / batch_size)
    # X_train = np.expand_dims(X_train, axis=3)
    # print(X_train.shape)

    # Create our Y for our Neural Networks
    valid = np.ones((batch_size, 1))
    fakes = np.zeros((batch_size, 1))

    for epoch in range(epochs):
      for j in range(bat_per_epo):
        # Get Random Batch
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Generate Fake Images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        # Train discriminator
        d_loss_real = discrimiator.train_on_batch(imgs, valid)
        d_loss_fake = discrimiator.train_on_batch(gen_imgs, fakes)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # inverse y label
        g_loss = GAN.train_on_batch(noise, valid)

        print("******** %d %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, j, d_loss[0], 100* d_loss[1], g_loss))

        #if(epoch % save_interval) == 0:
      save_imgs(epoch)
    
    # print(valid)
  
train(30000, batch_size=64, save_interval=200)

******** 0 0 [D loss: 0.021702, acc: 99.22%] [G loss: 0.481260]
******** 0 1 [D loss: 0.009510, acc: 100.00%] [G loss: 0.470114]
******** 0 2 [D loss: 0.033336, acc: 98.44%] [G loss: 0.464242]
******** 0 3 [D loss: 0.001308, acc: 100.00%] [G loss: 0.450176]
******** 0 4 [D loss: 0.042191, acc: 99.22%] [G loss: 0.441333]
******** 0 5 [D loss: 0.001981, acc: 100.00%] [G loss: 0.433809]
******** 0 6 [D loss: 0.003996, acc: 100.00%] [G loss: 0.426990]
******** 0 7 [D loss: 0.013271, acc: 100.00%] [G loss: 0.423956]
******** 0 8 [D loss: 0.006674, acc: 100.00%] [G loss: 0.416771]
******** 0 9 [D loss: 0.001329, acc: 100.00%] [G loss: 0.413816]
******** 0 10 [D loss: 0.008394, acc: 99.22%] [G loss: 0.408796]
******** 0 11 [D loss: 0.000263, acc: 100.00%] [G loss: 0.403910]
******** 0 12 [D loss: 0.000186, acc: 100.00%] [G loss: 0.397427]
******** 0 13 [D loss: 0.000213, acc: 100.00%] [G loss: 0.398254]
******** 0 14 [D loss: 0.002068, acc: 100.00%] [G loss: 0.393268]
******** 0 15 [D loss: 0